In [1]:
import pandas as pd

import os
import pickle as pkl

from sklearn.linear_model import Lasso, LassoCV, LogisticRegressionCV, LogisticRegression
from sklearn.svm import SVC
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, GridSearchCV, cross_validate, KFold, cross_val_predict
from sklearn.metrics import r2_score, explained_variance_score, normalized_mutual_info_score, \
    mutual_info_score, mean_absolute_error, make_scorer
from sklearn.dummy import DummyRegressor
from sklearn.utils.validation import check_X_y, check_array, check_is_fitted
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from sklearn.ensemble import IsolationForest

from skll.metrics import spearman, pearson 


import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap, Normalize
import seaborn as sns

from IPython.display import SVG,display,set_matplotlib_formats
set_matplotlib_formats('svg')

from PIL import Image

import warnings
warnings.filterwarnings('ignore')
from pylab import rcParams
rcParams['figure.figsize'] = 5,5

Custom model implementations and functions are stored in `src/dairyml.py`

In [2]:
from dairyml import PerfectClassifierMeanRegressor, plot_r2, BoundedLasso, BoundedLassoPlusLogReg, plot_coefficients, DummyRegressorCustom

## Import the Data
Load the data from the csv file, and set FoodCode to be the data index

In [3]:
data = pd.read_csv('../data/training_for_GS_122118.csv')
data = data.set_index('FoodCode')

Get the indices of all columns except description and lactose content - these columns are the numerical features that will be used as model input



In [4]:
numerical_features = [col for col in data.columns if col not in ['Food_Description','lac.per.100g']]
# print(numerical_features)

## Assess correlation of features with target variable

### Spearman's rank correlation coefficient

In [5]:
correlations = pd.DataFrame()
correlations['SRC'] = data[list(numerical_features) + ['lac.per.100g']].corr(method='spearman')['lac.per.100g']

### Mutual Information

In [6]:
for col in numerical_features:
    correlations.loc[col,'MI'] = mutual_info_score(data[col],data['lac.per.100g'])
    correlations.loc[col,'NMI'] = normalized_mutual_info_score(data[col],data['lac.per.100g'])

### Report average values for each correlation measure

In [7]:
correlations.abs().mean(axis=0)

SRC    0.204037
MI     3.192658
NMI    0.726827
dtype: float64

In [8]:
# correlations.to_excel('../data/correlations.xlsx')

## Prepare data for modelling
### Standardize the input features
- Use only the numerical features as model input `X`  
- Use sklearn's `StandardScaler` on these features: this standardizes features by removing the mean and scaling to unit variance 
- Convert the output of `StandardScaler` back to a dataframe for convenience

In [9]:
ss = StandardScaler()
ss.fit(data[numerical_features])
X = (data[numerical_features] - ss.mean_) / np.sqrt(ss.var_) #done by hand rather than fit_transform just to use same syntax as test script

Save mean and variance to file for use in test script

In [10]:
np.savetxt('../scaling/train_feature_means.csv',ss.mean_,delimiter=',')
np.savetxt('../scaling/train_feature_variances.csv',ss.var_,delimiter=',')

### Remove outliers

`contamination` parameter was hand-tuned based on the appearance of PCA & t-SNE visualizations below, increasing until data points that appeared to be outliers were removed

In [11]:
iso = IsolationForest(contamination=.013,random_state=7)
outliers = iso.fit_predict(X)
print('Outliers removed: {}'.format(sum(outliers == -1)))

Outliers removed: 5


### Use lactose as prediction target `Y`

In [12]:
Y = data['lac.per.100g']

In [13]:
X_in = X[outliers == 1]
Y_in = Y[outliers == 1]

In [14]:
data_outliers_removed = data.loc[outliers == 1,:]
data_outliers_removed.to_csv('../data/training_for_GS_122118_outliers_removed.csv')

## Distribution of target variable, class imbalance

In [15]:
Y_binary = (Y_in != 0).replace(True,'lactose').replace(False,'non-lactose')

#### Class absolute numbers

In [16]:
Y_binary.value_counts()

lactose        287
non-lactose     86
Name: lac.per.100g, dtype: int64

#### Class proportions

In [17]:
Y_binary.value_counts().divide(Y_binary.value_counts().sum()).round(2)

lactose        0.77
non-lactose    0.23
Name: lac.per.100g, dtype: float64

#### Distribution of all lactose values

In [18]:
ax = sns.distplot(Y_in,bins=20)
_ = ax.set_title("Distribution of all lactose values")
ax.figure.savefig('../graphics/lactose_distribution_all.png')

FileNotFoundError: [Errno 2] No such file or directory: '../graphics/lactose_distribution_all.png'

reload and save as tiff

In [ ]:
img = Image.open(open('../graphics/lactose_distribution_all.png','rb'))
img.save('../graphics/lactose_distribution_all.tiff',dpi=(300,300))

#### Distribution of nonzero lactose values

In [ ]:
ax = sns.distplot(Y_in[Y_in!=0],bins=20)
_ = ax.set_title("Distribution of nonzero lactose values")
ax.figure.savefig('../graphics/lactose_distribution_nonzero.png')

reload and save as tiff

In [ ]:
img = Image.open(open('../graphics/lactose_distribution_nonzero.png','rb'))
img.save('../graphics/lactose_distribution_nonzero.tiff',dpi=(300,300))

## Visualization: PCA & t-SNE

Create axes to plot visualization results

In [ ]:
f, axes = plt.subplots(2, 2, figsize=(10,5))

Create a colormap to outline the outliers

In [ ]:
red = np.array([212/256, 17/256, 17/256, 1])
nocolor = np.array([1,1,1,0])
colors = np.array([red,nocolor])

newcmp = ListedColormap(colors)

In [ ]:
newcmp

### PCA

Here we use PCA to visualize the features along two axes. Two plots are produced. In the first, the data points are colored by lactose value. In the second, the data points are colored by the logarithm of the lactose values. Since most lactose values are low, with a few high exceptions, coloring by lactose value directly obscures the differences on the lower end of the distribution, so coloring by log lactose attempts to fix that.  

In [ ]:
pca = PCA(n_components=2)
pca.fit(X_in)
pca_results = pd.DataFrame(pca.transform(X),columns=['PC1','PC2'],index=X.index)
pca_results['outlier_status'] = outliers

In [ ]:
pca_results['lac.per.100g'] = Y
pca_results['log.lac.per.100g'] = np.log(pca_results['lac.per.100g'] + .001)
pca_results['log_nonzero.lac.per.100g'] = np.log(pca_results['lac.per.100g'])
pca_results = pca_results.rename(columns={'lac.per.100g':'Lactose (g/100g)',
                             'log.lac.per.100g':'Log Lactose (g/100g)',
                             'log_nonzero.lac.per.100g':'Log Lactose (g/100g) (Nonzero)'})
# cmap = cm.get_cmap('Spectral')
ax = pca_results.plot.scatter('PC1','PC2',c='Log Lactose (g/100g) (Nonzero)',colormap='winter',ax=axes[0,0], edgecolors=newcmp(outliers))
axes[0,0].set_xlabel('PC1')
axes[0,0].set_title('PCA, colored by log lactose (nonzero only)')
plt.show()
ax = pca_results.plot.scatter('PC1','PC2',c='Log Lactose (g/100g)',colormap='winter',ax=axes[0,1], edgecolors=newcmp(outliers))
axes[0,1].set_xlabel('PC1')
axes[0,1].set_title('PCA, colored by log lactose')

### t-SNE

The same plots are created using t-SNE, a different visualization technique.

In [ ]:
tsne = TSNE(n_components=2,perplexity=30,random_state=7)
tsne_results = pd.DataFrame(tsne.fit_transform(X),columns=['axis 1','axis 2'],index=X.index)
tsne_results['outlier_status'] = outliers

In [ ]:
tsne_results['lac.per.100g'] = Y
tsne_results['log.lac.per.100g'] = np.log(tsne_results['lac.per.100g'] + .001)
tsne_results['log_nonzero.lac.per.100g'] = np.log(tsne_results['lac.per.100g'])
tsne_results = tsne_results.rename(columns={'lac.per.100g':'Lactose (g/100g)',
                             'log.lac.per.100g':'Log Lactose (g/100g)',
                             'log_nonzero.lac.per.100g':'Log Lactose (g/100g) (Nonzero)'})

ax = tsne_results.plot.scatter('axis 1','axis 2',c='Log Lactose (g/100g) (Nonzero)',colormap='winter',ax=axes[1,0], edgecolors=newcmp(outliers))
axes[1,0].set_xlabel('axis 1')
axes[1,0].set_title('t-SNE, colored by log lactose (nonzero only)')

ax = tsne_results.plot.scatter('axis 1','axis 2',c='Log Lactose (g/100g)',colormap='winter',ax=axes[1,1], edgecolors=newcmp(outliers))
axes[1,1].set_xlabel('axis 1')
axes[1,1].set_title('t-SNE, colored by log lactose')

In [ ]:
f.tight_layout()

In [ ]:
f

In [ ]:
f.savefig('../graphics/pca_tsne_plots.png')

reload and save as tiff

In [ ]:
img = Image.open(open('../graphics/pca_tsne_plots.png','rb'))
img.save('../graphics/pca_tsne_plots.tiff',dpi=(300,300))

Export pca and tsne results to file

In [ ]:
pca_results.to_csv('../graphics/pca_results.csv')

In [ ]:
tsne_results.to_csv('../graphics/tsne_results.csv')

### Save preprocessed data to pickle files

In [ ]:
data_to_save = [X_in, Y_in]
data_dir = '../pkl/data/'
try:
    os.makedirs(data_dir)
except FileExistsError:
    pass

save_string = "data_outliers_removed"

with open(data_dir + save_string, "wb" ) as f:
    pkl.dump(data_to_save,f)

print('saved to {}'.format(data_dir + save_string))